In [1]:
import scipy.io
import matplotlib.pyplot as plt
import torch
from torch import tensor
import numpy as np
from RNO_1D import *
from tqdm import tqdm

# Load the .mat file
file_path = r'C:\Users\kiril\OneDrive\Documents\University\IIB\4C11 Data-Driven and Learning-Based Methods in Mechanincs and Materials\Coursework3\viscodata_3mat.mat'
data = scipy.io.loadmat(file_path)
matread = MatReader(file_path)

In [2]:
sigma = tensor(data['sigma_tol']).unsqueeze(1).float()
eps = tensor(data['epsi_tol']).unsqueeze(1).float()
sigma.shape, eps.shape, sigma.dtype, eps.dtype# [batch, channel, time]

(torch.Size([400, 1, 1001]),
 torch.Size([400, 1, 1001]),
 torch.float32,
 torch.float32)

In [3]:
from torch.utils.data import TensorDataset, random_split, DataLoader

# Combine sigma and eps tensors into a single dataset
dataset = TensorDataset(eps, sigma)

# Define the size of train and test datasets
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size

# Split the dataset into train and test datasets
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Print the size of train and test datasets
print(f'Train dataset size: {len(train_dataset)}')
print(f'Test dataset size: {len(test_dataset)}')

Train dataset size: 280
Test dataset size: 120


In [4]:
batch_size = 280
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size)

In [5]:
eps.shape

torch.Size([400, 1, 1001])

In [ ]:
net = RNO(17, 1, 32, 5)
h = torch.zeros(16)

x = eps[:7, :, 0]
print(x.shape)
y = net(x, 1e-3)
print(y.shape)

torch.Size([1])


TypeError: RNO.forward() missing 1 required positional argument: 'dt'

In [ ]:
width = 64
depth = 3
hidden_size = 16
net = RNO(1, width, depth, hidden_size, batch_size)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

In [ ]:
len_traj = 1001
n_epochs = 50
loss = torch.nn.MSELoss()

In [ ]:
training_losses = []
testing_losses = []
for _ in tqdm(range(5)):
    train_loss = 0.0
    for input, target in train_loader:
        optimizer.zero_grad()
        h = torch.zeros(batch_size, hidden_size)
        for i in range(len_traj):
            y = net(input[:,:,i], 1e-3)
            l = loss(y, target[:,i])
            l.backward()
            optimizer.step()

  0%|          | 0/5 [00:00<?, ?it/s]c:\Users\kiril\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([280, 1001])) that is different to the input size (torch.Size([280, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  0%|          | 0/5 [00:00<?, ?it/s]


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

num_params = count_parameters(net)
print(f'The number of trainable parameters in the network: {num_params}')

The number of trainable parameters in the network: 20049
